In [ ]:
# upload audio file
from google.colab import files
uploaded = files.upload()
path = next(iter(uploaded))

Saving resistance.wav to resistance.wav


In [ ]:
num_speakers = 2 #@param {type:"integer"}

language = 'any' #@param ['any', 'English']

model_size = 'large' #@param ['tiny', 'base', 'small', 'medium', 'large']


model_name = model_size
if language == 'English' and model_size != 'large':
  model_name += '.en'


In [ ]:
!pip install pydub

In [ ]:
!pip install -q git+https://github.com/openai/whisper.git > /dev/null
!pip install -q git+https://github.com/pyannote/pyannote-audio > /dev/null

import whisper
import datetime

import subprocess

import torch
import pyannote.audio
from pyannote.audio.pipelines.speaker_verification import PretrainedSpeakerEmbedding
embedding_model = PretrainedSpeakerEmbedding(
    "speechbrain/spkrec-ecapa-voxceleb",
    device=torch.device("cuda"))

from pyannote.audio import Audio
from pyannote.core import Segment
from pydub import AudioSegment

import wave
import contextlib

from sklearn.cluster import AgglomerativeClustering
import numpy as np

hyperparams.yaml:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

embedding_model.ckpt:   0%|          | 0.00/83.3M [00:00<?, ?B/s]

mean_var_norm_emb.ckpt:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

classifier.ckpt:   0%|          | 0.00/5.53M [00:00<?, ?B/s]

label_encoder.txt:   0%|          | 0.00/129k [00:00<?, ?B/s]

In [ ]:
!pip install pydub
!apt-get install ffmpeg


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [ ]:
model = whisper.load_model(model_size)

100%|█████████████████████████████████████| 2.88G/2.88G [01:00<00:00, 50.7MiB/s]


In [ ]:
result = model.transcribe(path)
segments = result["segments"]

In [ ]:
print(segments)

[{'id': 0, 'seek': 0, 'start': 0.0, 'end': 2.0, 'text': " So you're with the Resistance?", 'tokens': [50365, 407, 291, 434, 365, 264, 45647, 30, 50465], 'temperature': 0.0, 'avg_logprob': -0.26358009639539215, 'compression_ratio': 1.4580645161290322, 'no_speech_prob': 0.2863471806049347}, {'id': 1, 'seek': 0, 'start': 2.0, 'end': 6.0, 'text': ' 🎵', 'tokens': [50465, 25674, 50665], 'temperature': 0.0, 'avg_logprob': -0.26358009639539215, 'compression_ratio': 1.4580645161290322, 'no_speech_prob': 0.2863471806049347}, {'id': 2, 'seek': 0, 'start': 6.0, 'end': 8.0, 'text': ' Obviously.', 'tokens': [50665, 7580, 13, 50765], 'temperature': 0.0, 'avg_logprob': -0.26358009639539215, 'compression_ratio': 1.4580645161290322, 'no_speech_prob': 0.2863471806049347}, {'id': 3, 'seek': 0, 'start': 8.0, 'end': 10.0, 'text': ' Yes, I am.', 'tokens': [50765, 1079, 11, 286, 669, 13, 50865], 'temperature': 0.0, 'avg_logprob': -0.26358009639539215, 'compression_ratio': 1.4580645161290322, 'no_speech_prob':

In [ ]:
audio = Audio()
audio_segment = AudioSegment.from_file(path)
audio_segment.export('audio.wav', format='wav')
path = 'audio.wav'
# Get audio duration using pydub
duration = len(audio_segment) / 1000.0

In [ ]:


def segment_embedding(segment):
  start = segment["start"]
  # Whisper overshoots the end timestamp in the last segment
  end = min(duration, segment["end"])
  clip = Segment(start, end)
  waveform, sample_rate = audio.crop(path, clip)
  # Ensure single channel (mono)
  if waveform.shape[0] > 1:
      waveform = waveform.mean(axis=0, keepdims=True)
  return embedding_model(waveform[None])

In [ ]:
embeddings = np.zeros(shape=(len(segments), 192))
for i, segment in enumerate(segments):
  embeddings[i] = segment_embedding(segment)

embeddings = np.nan_to_num(embeddings)

In [ ]:
clustering = AgglomerativeClustering(num_speakers).fit(embeddings)
labels = clustering.labels_
for i in range(len(segments)):
  segments[i]["speaker"] = 'SPEAKER ' + str(labels[i] + 1)

In [ ]:
def time(secs):
  return datetime.timedelta(seconds=round(secs))

f = open("transcript.txt", "w")

for (i, segment) in enumerate(segments):
  if i == 0 or segments[i - 1]["speaker"] != segment["speaker"]:
    f.write("\n" + segment["speaker"] + ' ' + str(time(segment["start"])) + '\n')
  f.write(segment["text"][1:] + ' ')
f.close()